# P8.4 Azure segmentation model registration

## 1.Setting up working environment

In [2]:
import segmentation_models as sm
sm.set_framework('tf.keras')

Segmentation Models: using `keras` framework.


In [3]:
import tensorflow as tf

In [4]:
import azureml.core
from azureml.core import Workspace, Dataset
# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, sep='\t')

Azure ML SDK Version:  1.39.0
docs-ws	francecentral	openclassrooms


## 2. Segmentation model loading

In [5]:
# define network parameters
n_classes = 9
activation = 'softmax'
BACKBONE = 'efficientnetb3' #'mobilenetv2'
preprocess_input = sm.get_preprocessing(BACKBONE)

#create model
model = sm.PSPNet(BACKBONE,encoder_weights='imagenet',encoder_freeze=True, classes=n_classes, activation=activation)
model.summary()

44122112/44107200 [==============================] - 1s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 384, 384, 3  0           []                               
                                )]                                                                
                                                                                                  
 stem_conv (Conv2D)             (None, 192, 192, 40  1080        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 stem_bn (BatchNormalization)   (None, 192, 192, 40  160         ['stem_conv[0][0]']              
                             

In [14]:
model.load_weights('outputs/pspnet_train.h5')

In [15]:
model.save('outputs/pspnet_train')

INFO:tensorflow:Assets written to: outputs/pspnet_train/assets


## 3. Model registrer 

In [16]:
   from azureml.core import Model
   from azureml.core.resource_configuration import ResourceConfiguration


   model = Model.register(workspace=ws,
                          model_name='pspnet_model',                # Name of the registered model in your workspace.
                          model_path='outputs/pspnet_train',  # Local file to upload and register as a model.
                          resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=2),
                          description='semantic segmentation model for driving scenes',
                          tags={'area': 'roads', 'type': 'semantic segmentation'})

   print('Name:', model.name)
   print('Version:', model.version)

Registering model pspnet_model
Name: pspnet_model
Version: 3
